In [1]:
#Import libraries
from sklearn import preprocessing
%matplotlib inline
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import math
import operator
import seaborn as sns
sns.set_style('white')

#ml algos
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

pd.options.display.max_rows = 30

In [2]:
#import csv for Austin B-Cycle Trip data
bcycle = pd.read_csv('Austin_B-Cycle_Trips.csv', encoding='latin1')

#import csv for private dockless rideshare co's
dockless = pd.read_csv('Dockless_Vehicle_Trips.csv', encoding='latin1')

/Users/jakegrosek/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/jakegrosek/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
num_trips = len(bcycle)
print("There are {} trips in this dataset".format(num_trips))

There are 1171610 trips in this dataset


In [4]:
# data clean columns: 
bcycle.columns = bcycle.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')

## We can drop Year and Month since ~50% of the values in both colummns are NaN while the Checkout Date are 100% filled.

In [5]:
bcycle.drop(['month', 'year'], 1, inplace=True)

In [7]:
bcycle.head(1)

,trip_id,membership_type,bicycle_id,checkout_date,checkout_time,checkout_kiosk_id,checkout_kiosk,return_kiosk_id,return_kiosk,trip_duration_minutes
0,9900285854,Annual (San Antonio B-cycle),207,10/26/2014,13:12:00,2537.0,West & 6th St.,2707.0,Rainey St @ Cummings,76


## bicycle_id column cleaning section

In [9]:
bcycle.loc[1077930]

trip_id                                 19000898
membership_type          U.T. Student Membership
bicycle_id                                  303G
checkout_date                         11/18/2018
checkout_time                           22:35:50
checkout_kiosk_id                           2547
checkout_kiosk                  Guadalupe & 21st
return_kiosk_id                             3838
return_kiosk                       Nueces & 26th
trip_duration_minutes                          7
Name: 1077930, dtype: object

In [43]:
bcycle.loc[bcycle.bicycle_id == '303G', 'bicycle_id'].count()

225

In [51]:
bcycle.loc[bcycle.bicycle_id == '303', 'bicycle_id'].count()

225

In [45]:
bcycle.loc[bcycle.bicycle_id == '134G', 'bicycle_id'].count()

377

In [46]:
bcycle.loc[bcycle.bicycle_id == '134', 'bicycle_id'].count()

227

In [109]:
bcycle = bcycle['bicycle_id'] != '303G')

IndexError: bicycle_id

In [39]:
pd.to_numeric(bcycle['bicycle_id'], downcast='integer')

ValueError: Unable to parse string "303G" at position 1077930

## return_ and checkout_kiosk_id column cleaning section

# inplace wont work?

In [82]:
bcycle.isnull().sum()

trip_id                      0
membership_type           6463
bicycle_id                 723
checkout_date                0
checkout_time                0
checkout_kiosk_id        23154
checkout_kiosk               0
return_kiosk_id          24423
return_kiosk                 0
trip_duration_minutes        0
dtype: int64

In [95]:
bcycle['bicycle_id'].dropna(inplace=True)

In [96]:
bcycle['membership_type'].dropna(inplace=True)

In [97]:
bcycle.isnull().sum()

trip_id                      0
membership_type           6463
bicycle_id                 723
checkout_date                0
checkout_time                0
checkout_kiosk_id        23154
checkout_kiosk               0
return_kiosk_id          24423
return_kiosk                 0
trip_duration_minutes        0
dtype: int64

In [79]:
bcycle_test = bcycle[['checkout_kiosk_id', 'return_kiosk_id', ]].dropna(axis=0, how='any',inplace=False)

,checkout_kiosk_id,return_kiosk_id
0,2537.0,2707.0
1,2498.0,2566.0
2,2537.0,2496.0
6,2496.0,2561.0
7,2569.0,2499.0
8,2496.0,2496.0
10,2504.0,2496.0
12,2575.0,2575.0
13,2494.0,2570.0
14,2504.0,2540.0


In [76]:
bcycle['checkout_kiosk_id'] = bcycle['checkout_kiosk_id'].astype(int)



ValueError: Cannot convert non-finite values (NA or inf) to integer

In [72]:
pd.to_numeric(bcycle['return_kiosk_id'], downcast='integer')


0          2707.0
1          2566.0
2          2496.0
3             NaN
4             NaN
5          2545.0
6          2561.0
7          2499.0
8          2496.0
9          2545.0
10         2496.0
11            NaN
12         2575.0
13         2570.0
14         2540.0
            ...  
1171595    2574.0
1171596    3792.0
1171597    2566.0
1171598    2552.0
1171599    3841.0
1171600    2572.0
1171601    3797.0
1171602    3793.0
1171603    4061.0
1171604    3792.0
1171605    3660.0
1171606    2707.0
1171607    3684.0
1171608    2540.0
1171609    2542.0
Name: return_kiosk_id, Length: 1171610, dtype: float64

In [73]:
bcycle.dtypes

trip_id                           int64
membership_type                  object
bicycle_id                       object
checkout_date            datetime64[ns]
checkout_time                    object
checkout_kiosk_id               float64
checkout_kiosk                   object
return_kiosk_id                 float64
return_kiosk                     object
trip_duration_minutes             int64
dtype: object

In [25]:
bcycle.dtypes

trip_id                           int64
membership_type                  object
bicycle_id                       object
checkout_date            datetime64[ns]
checkout_time                    object
checkout_kiosk_id               float64
checkout_kiosk                   object
return_kiosk_id                 float64
return_kiosk                     object
trip_duration_minutes             int64
dtype: object

In [13]:
bcycle['checkout_date'] = pd.to_datetime(bcycle['checkout_date'])

## There is also a mismatch betweent the number of unique Return & Checkout Kiosk and the corresponding ID's

In [14]:
bcycle['checkout_kiosk_id'].nunique()

96

In [15]:
bcycle['checkout_kiosk'].nunique()

120

In [17]:
bcycle['return_kiosk_id'].nunique()

96

In [18]:
bcycle['return_kiosk'].nunique()

123

In [ ]:
return_k = bcycle['Return Kiosk'].value_counts()
len(return_k)

## While half of the Months and Years are missing, fortunately none of the values are missing in Checkout Date--we can replace

In [111]:
bcycle.head()

0    True
1    True
2    True
3    True
4    True
Name: bicycle_id, dtype: bool

In [4]:
bcycle.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1171610 entries, 0 to 1171609
Data columns (total 12 columns):
Trip ID                  1171610 non-null int64
Membership Type          1165147 non-null object
Bicycle ID               1170887 non-null object
Checkout Date            1171610 non-null object
Checkout Time            1171610 non-null object
Checkout Kiosk ID        1148456 non-null float64
Checkout Kiosk           1171610 non-null object
Return Kiosk ID          1147187 non-null float64
Return Kiosk             1171610 non-null object
Trip Duration Minutes    1171610 non-null int64
Month                    618479 non-null float64
Year                     618479 non-null float64
dtypes: float64(4), int64(2), object(6)
memory usage: 498.6 MB


things to reduce
bicycle id-> int 8
checkout ID-> int 16
return ID-> int 16
trip duration-> int 16
month-> int 8
year-> int 16

get dummies
membership
checkout kiosk
return kiosk

In [27]:
dockless.head(10)

,id,device_id,vehicle_type,trip_duration,trip_distance,start_time,end_time,modified_date,month,hour,day_of_week,council_district_start,council_district_end,year,census_tract_start,census_tract_end
0,6865731e-5e4b-4819-9435-4e62e09a652d,08c8c852-257c-49ad-a14f-727bfc79626d,NaN,NaN,NaN,NaN,NaN,12/13/2018 03:36:12 PM,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN
1,08dc0d9c-735e-4006-b9af-60011509e469,255716b5-4a9f-4efc-9967-7a65b3521bac,bicycle,11491603.0,1658.0,11/30/2018 11:00:00 AM,04/12/2019 12:00:00 PM,12/21/2018 12:41:42 AM,11.0,11.0,5.0,9.0,0.0,2018.0,NaN,NaN
2,aa27d854-9f27-456e-ae49-f268a9b2b533,6b90a827-60a1-47ad-94a4-d33654dbd50d,scooter,170.0,0.0,04/29/2019 05:30:00 PM,04/29/2019 05:30:00 PM,04/30/2019 06:45:17 AM,4.0,17.0,1.0,9.0,9.0,2019.0,4.8453e+10,48453000601
3,b7c9f133-966b-46f1-91ce-2682ce7fcf50,1e5234a3-e86b-41e1-a1ad-e98310f3a71c,scooter,90.0,0.0,04/29/2019 05:30:00 PM,04/29/2019 05:30:00 PM,04/30/2019 06:45:17 AM,4.0,17.0,1.0,9.0,9.0,2019.0,4.8453e+10,48453000601
4,1d4f5d89-c044-46cb-8674-80d37fa0371f,caa0325c-7c0b-4909-a573-d83126ecc953,scooter,570.0,826.0,04/29/2019 05:15:00 PM,04/29/2019 05:15:00 PM,04/30/2019 06:45:17 AM,4.0,17.0,1.0,9.0,9.0,2019.0,4.8453e+10,48453001100
5,cd442fb9-058f-46a7-af90-b4adb56fd163,1653cf10-75f5-4934-a9d9-c8d43ade67a9,scooter,438.0,1815.0,04/29/2019 05:30:00 PM,04/29/2019 05:45:00 PM,04/30/2019 06:45:17 AM,4.0,17.0,1.0,9.0,9.0,2019.0,4.8453e+10,48453000500
6,e1ac0e42-ab2d-4eac-b019-f085169c1d38,1d89f9e2-d01a-4736-b7d8-258cb9a2108c,scooter,391.0,1519.0,04/29/2019 05:30:00 PM,04/29/2019 05:30:00 PM,04/30/2019 06:45:17 AM,4.0,17.0,1.0,9.0,9.0,2019.0,4.8453e+10,48453000307
7,b7df9ed5-61ae-40ca-9b7e-0ffa1d092b79,09b2c7b8-6631-4999-85d5-4273b4c98f3a,scooter,153.0,0.0,04/29/2019 05:00:00 PM,04/29/2019 05:00:00 PM,04/30/2019 06:45:17 AM,4.0,17.0,1.0,9.0,9.0,2019.0,4.8453e+10,48453000604
8,1c8b3eab-6aed-4b69-8eca-6b5e0b70a45f,ab262134-2421-49b7-a97f-fadde23e71db,scooter,570.0,1843.0,04/29/2019 05:30:00 PM,04/29/2019 05:45:00 PM,04/30/2019 06:45:17 AM,4.0,17.0,1.0,9.0,9.0,2019.0,4.8453e+10,48453001100
9,f9174176-9d16-431d-af21-ebcab226da31,73e8b0bf-3aae-4f80-8683-c5d7c3b3989a,scooter,668.0,2829.0,04/29/2019 05:15:00 PM,04/29/2019 05:30:00 PM,04/30/2019 06:45:17 AM,4.0,17.0,1.0,9.0,9.0,2019.0,4.8453e+10,48453001200


In [7]:
dockless.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5442039 entries, 0 to 5442038
Data columns (total 16 columns):
ID                          object
Device ID                   object
Vehicle Type                object
Trip Duration               float64
Trip Distance               float64
Start Time                  object
End Time                    object
Modified Date               object
Month                       float64
Hour                        float64
Day of Week                 float64
Council District (Start)    float64
Council District (End)      float64
Year                        float64
Census Tract Start          object
Census Tract End            object
dtypes: float64(8), object(8)
memory usage: 3.5 GB


## I did a quick calculation of the average memory usage per column to verify...and it's a lot

In [10]:
# memory calc for B cycles data set
for dtype in ['float','int','object']:
    selected_dtype = bcycle.select_dtypes(include=[dtype])
    mean_usage_b = selected_dtype.memory_usage(deep=True).mean()
    mean_usage_mb = mean_usage_b / 1024 ** 2
print("Average memory usage for {} columns: {:03.2f}MB".format(dtype,mean_usage_mb))

Average memory usage for object columns: 63.57MB


In [11]:
# memory calc for dockless data set
for dtype in ['float','int','object']:
    selected_dtype = dockless.select_dtypes(include=[dtype])
    mean_usage_b = selected_dtype.memory_usage(deep=True).mean()
    mean_usage_mb = mean_usage_b / 1024 ** 2
print("Average memory usage for {} columns: {:03.2f}MB".format(dtype,mean_usage_mb))

Average memory usage for object columns: 356.86MB


In [13]:
int_types = ["uint8", "int8", "int16"]
for it in int_types:
    print(np.iinfo(it))

Machine parameters for uint8
---------------------------------------------------------------
min = 0
max = 255
---------------------------------------------------------------

Machine parameters for int8
---------------------------------------------------------------
min = -128
max = 127
---------------------------------------------------------------

Machine parameters for int16
---------------------------------------------------------------
min = -32768
max = 32767
---------------------------------------------------------------



In [38]:
# Created a function to save time
 
def mem_usage(pandas_obj):
    if isinstance(pandas_obj,pd.DataFrame):
        usage_b = pandas_obj.memory_usage(deep=True).sum()
    else: # we assume if not a df it's a series 
        usage_b = pandas_obj.memory_usage(deep=True)
        # convert bytes to megabytes
    usage_mb = usage_b / 1024 ** 2
    return usage_mb

In [39]:
bcycle_int = bcycle.select_dtypes(include=['int'])
converted_int = bcycle_int.apply(pd.to_numeric,downcast='unsigned')
print(mem_usage(bcycle_int))
print(mem_usage(converted_int))
compare_ints = pd.concat([bcycle_int.dtypes,converted_int.dtypes],axis=1)
compare_ints.columns = ['before','after']
compare_ints.apply(pd.Series.value_counts)

26.65155029296875
19.988662719726562


,before,after
uint16,NaN,1.0
int64,2.0,NaN
uint64,NaN,1.0


In [42]:
bcycle_int?

In [40]:
bcycle.dtypes

trip_id                           int64
membership_type                  object
bicycle_id                       object
checkout_date            datetime64[ns]
checkout_time                    object
checkout_kiosk                   object
return_kiosk                     object
trip_duration_minutes             int64
dtype: object

In [24]:
bcycle_float = bcycle.select_dtypes(include=['float'])
converted_float = bcycle_float.apply(pd.to_numeric,downcast='float')
print(mem_usage(bcycle_float))
print(mem_usage(converted_float))
compare_floats = pd.concat([bcycle_float.dtypes,converted_float.dtypes],axis=1)
compare_floats.columns = ['before','after']
compare_floats.apply(pd.Series.value_counts)

UnboundLocalError: local variable 'usage_mb' referenced before assignment

# Dockless Section

ID - arbitrary, per website
Device ID - arbitrary, per website
trip - seconds and in meters
month - jan = 1, etc
day - sunday = 0, etc
council district - 


to do:
- get dummies/recode ID and Device ID for easier management?
- get dummies on cehicle type
- convert trip to minutes?
- drop two census columns and modified date
- covert duration?, distance, year to int 16 
- convert month, hour, day, council start/finish,  to int 8
- what to do w/ start/end time?

In [67]:
num_trips = len(dockless)
print("There are {} trips in this dataset".format(num_trips))

There are 5442039 trips in this dataset


In [24]:
# data clean columns: 
dockless.columns = dockless.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')

In [25]:
dockless.sample(5)

,id,device_id,vehicle_type,trip_duration,trip_distance,start_time,end_time,modified_date,month,hour,day_of_week,council_district_start,council_district_end,year,census_tract_start,census_tract_end
1909719,12a6f192-5ae4-41ce-90b6-158b8b6c283f,23d6137d-fbbc-4139-a96e-db4af628c581,scooter,200.0,338.0,09/27/2018 09:30:00 AM,09/27/2018 09:45:00 AM,04/17/2019 07:02:28 AM,9.0,9.0,4.0,9.0,9.0,2018.0,48453001100,48453001100
2834537,d715bc93-cb77-4066-b4e9-e54986342970,d56a12f8-3a6d-42d4-a475-9f40e7355216,scooter,700.0,0.0,11/09/2018 04:15:00 PM,11/09/2018 04:30:00 PM,04/17/2019 02:34:24 AM,11.0,16.0,5.0,9.0,9.0,2018.0,48453001100,48453001401
3278795,bc1b6c28-27d0-4ad3-a1a6-06f0718776a9,e3eb3352-748e-4e74-9958-69d2d5e27b41,scooter,449.0,1823.0,04/23/2019 03:00:00 PM,04/23/2019 03:15:00 PM,04/24/2019 10:38:04 PM,4.0,15.0,2.0,9.0,9.0,2019.0,48453000601,48453000401
2855068,b59c11cf-7b4d-47fe-89a6-1cc2f560c465,988f68c6-d18c-4c1a-8bd8-0136c6fbd396,scooter,445.0,2381.0,04/09/2019 08:00:00 AM,04/09/2019 08:15:00 AM,04/17/2019 02:36:31 AM,4.0,8.0,2.0,7.0,10.0,2019.0,48453000205,48453000204
4066781,2d03782c-f4b7-428d-815c-e0bc0f8fc056,418f89a6-0289-4bf1-a194-2ee59153d447,scooter,726.0,2012.0,11/04/2018 09:15:00 AM,11/04/2018 09:30:00 AM,04/17/2019 05:18:42 AM,11.0,9.0,0.0,3.0,3.0,2018.0,48453001000,48453000902


In [69]:
dockless.isnull().sum()

ID                            0
Device ID                     0
Vehicle Type                  1
Trip Duration                 1
Trip Distance                 1
Start Time                    1
End Time                      1
Modified Date                 0
Month                         1
Hour                          1
Day of Week                   1
Council District (Start)    186
Council District (End)      186
Year                          1
Census Tract Start          188
Census Tract End            188
dtype: int64

## For the most part, all the trips start and end in the same Council District, with the exception being zone Zero starting at 5th place but ending at 8th.

In [70]:
dockless['Council District (Start)'].value_counts()

9.0     4262556
3.0      498504
1.0      327468
5.0      189368
0.0       41657
7.0       37811
8.0       33117
10.0      28544
4.0       16669
2.0        5672
6.0         487
Name: Council District (Start), dtype: int64

In [71]:
dockless['Council District (End)'].value_counts()

9.0     4191987
3.0      514803
1.0      362081
5.0      203877
7.0       38961
8.0       38768
10.0      32538
0.0       31676
4.0       19330
2.0        7348
6.0         484
Name: Council District (End), dtype: int64